# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
#Read the information from cities.csv and output in data frame

city_data = pd.read_csv('output_data/cities.csv')

city_data.set_index("City ID", inplace=True)

city_data.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
City ID,,,,,,,,,
0,Cape Town,ZA,-33.9258,18.4232,67.08,80,67,1.66,2022-02-08 17:07:14-06:00
1,Busselton,AU,-33.6500,115.3333,56.05,76,3,6.11,2022-02-08 17:07:16-06:00
2,Port Elizabeth,ZA,-33.9180,25.5701,62.89,94,20,9.22,2022-02-08 17:07:16-06:00
3,Ribeira Grande,PT,38.5167,-28.7000,60.35,70,93,5.95,2022-02-08 17:07:16-06:00
4,Cayenne,GF,4.9333,-52.3333,80.10,84,20,4.00,2022-02-08 17:07:16-06:00


## Formatting maps

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Formatting for both the 'humidity heat map' and the 'hotel location map'
figure_layout = {
    'width': '900px',
    'height': '475px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

map_center = (21, 0)
zoom = 1.65

# Heat Map for Humidity Level

In [5]:
#Set the Latitude and Longitude as locations and Humidity as the weight for the heat map
locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity (%)"]

#Set up empty map
fig = gmaps.figure(center = map_center, zoom_level = zoom, layout=figure_layout)

#Add heatmap layer to map
heatmap_layer = gmaps.heatmap_layer(locations, weight, dissipating = True, max_intensity = 110, point_radius = 8)
fig.add_layer(heatmap_layer)

#Display map
fig

Figure(layout=FigureLayout(border='1px solid black', height='475px', margin='0 auto 0 auto', padding='1px', wi…

## Hotel Map

In [6]:
#Narrow down data frame to ideal vacation conditions
vacay_temp_min = 75.0
vacay_temp_max = 87.5
wind_speed_max = 10.0
humidity_max = 75
cloud_max = 20.0

#Create data frame from only the ideal vacation locations
vacation_cities = city_data[(city_data["Max Temp (F)"] >= vacay_temp_min) & 
                            (city_data["Max Temp (F)"] < vacay_temp_max) &
                            (city_data["Wind Speed (mph)"] <= wind_speed_max) &
                            (city_data["Humidity (%)"] < humidity_max) &
                            (city_data["Cloudiness (%)"] < cloud_max)                           
                           ]

#Confirm that the "ideal" parameters are not too narrow
print(f" {len(vacation_cities.index)} cities match vacation parameters.")

 20 cities match vacation parameters.


In [7]:
# Find closest hotel/lodging

#Create a data frame that will include hotel inforamtion
hotel_df = vacation_cities.copy()

#Add empty columns for "Hotel Name" and "Hotel Rating" to data frame
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Rating"] = ""

# Base url for gmaps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up parameter dictionary. Note: "lodging" is listed as a "Place Type" in gmaps API documentation, "hotel" is not
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

#Loop through cities in the hotel_df data frame to look for hotels
for index, row in hotel_df.iterrows():

    #Use the city's latitude and longitude to find the nearest hotel/lodging
    city_lat = row["Latitude"]
    city_lon = row["Longitude"]

    #append lat, lon to list of parameters
    params["location"] = f"{city_lat},{city_lon}"
    
    #get data from gmaps
    response = requests.get(base_url, params = params).json()  

    try:       
        #Add first hotel result for each city to data frame        
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Rating"] = response["results"][0]["rating"]
        print(f"Retrieving results for city {index}: {row[0]}")
        
    except (KeyError, IndexError):
        print(f"Missing field. Skipping {row[0]}.")

#Drop data frame rows with blanks
hotel_df = hotel_df[(hotel_df["Hotel Name"] != '') & (hotel_df["Hotel Rating"] != '')]   

#Convert ratings from objects to numbers
hotel_df["Hotel Rating"] = pd.to_numeric(hotel_df["Hotel Rating"], errors = 'coerce')

#Only keep the top 10 rated hotels, ensures that no more than 10 hotels are pinned on the map
hotel_df = hotel_df.nlargest(10, "Hotel Rating", keep = 'first')

#Drop current weather information to simplify data frame
hotel_df = hotel_df.drop(["Max Temp (F)", "Humidity (%)", "Cloudiness (%)", "Wind Speed (mph)", "Date"], axis = 1)

#Display the data frame
hotel_df

Retrieving results for city 17: Lompoc
Retrieving results for city 38: Flinders
Retrieving results for city 50: Formosa
Retrieving results for city 100: Charlestown
Retrieving results for city 168: Patterson
Retrieving results for city 178: Cabo San Lucas
Retrieving results for city 187: Laguna
Retrieving results for city 199: Mount Isa
Retrieving results for city 213: Champerico
Retrieving results for city 219: Mackay
Retrieving results for city 228: Port Macquarie
Retrieving results for city 263: Todos Santos
Retrieving results for city 276: San Luis
Retrieving results for city 292: Villa Hayes
Retrieving results for city 306: Pacific Grove
Retrieving results for city 314: Reconquista
Retrieving results for city 408: Batemans Bay
Retrieving results for city 436: Half Moon Bay
Retrieving results for city 538: La Libertad
Retrieving results for city 565: Rodolfo Sánchez Taboada


,City,Country,Latitude,Longitude,Hotel Name,Hotel Rating
City ID,,,,,,
292,Villa Hayes,PY,-25.0931,-57.5236,Dirección chinos villa hayes,5.0
219,Mackay,AU,-21.1500,149.2000,International Lodge Motel,4.6
436,Half Moon Bay,US,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay",4.6
538,La Libertad,EC,-2.2333,-80.9000,Playa Canela Hotel Boutique,4.6
263,Todos Santos,MX,23.4500,-110.2167,Hotel California Todos Santos,4.5
178,Cabo San Lucas,MX,22.8909,-109.9124,Hotel Tesoro Los Cabos,4.4
187,Laguna,US,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...,4.3
17,Lompoc,US,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel",4.2
213,Champerico,GT,14.3000,-91.9167,"Hotel y restaurante ""El Diamante""",4.2


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Hotel Rating</dt><dd>{Hotel Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [9]:
#Hover text information
hotel_name = hotel_df["Hotel Name"].astype(str)
hotel_name.reset_index(drop = True, inplace = True)

In [11]:
hotel_locations = hotel_df[["Latitude", "Longitude"]]

#Set up empty map
fig = gmaps.figure(layout = figure_layout, center = map_center, zoom_level = zoom)

#Add heat map layer
heatmap_layer = gmaps.heatmap_layer(locations, weight, dissipating = False, max_intensity = 110, point_radius = 3)
fig.add_layer(heatmap_layer)

#Add layer with hotel markers
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info, hover_text = hotel_name)
fig.add_layer(marker_layer)

#Display the figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='475px', margin='0 auto 0 auto', padding='1px', wi…